### Naive Bayes From Scratch

In [3]:
import numpy as np

In [ ]:
class GaussianNaiveBayes:
    def __init__(self):
        """
        Initializes empty dictionaries to store:
        - priors: probability of each class P(C)
        - means: mean of each feature for each class
        - variances: variance of each feature for each class
        """
        self.class_priors = {}  # P(C) for each class
        self.means = {}        # Mean of features per class
        self.variances = {}    # Variance of features per class



    def fit(self, X, y):
        """
        Trains the Naive Bayes classifier.
        Parameters:
        X : numpy array of shape (m, n) - Training features
        y : numpy array of shape (m,)   - Training labels
        """
        m, n = X.shape  # m: number of samples, n: number of features
        classes = np.unique(y)  # All unique class labels
        
        for cls in classes:
            # Get all rows where label == cls
            X_cls = X[y == cls]
            
            # Prior probability P(C) = (# samples in class) / (total samples)
            self.class_priors[cls] = X_cls.shape[0] / m
            
            # Mean and variance for each feature in this class
            self.means[cls] = np.mean(X_cls, axis=0)
            self.variances[cls] = np.var(X_cls, axis=0)        



    def gaussian_probability(self, x, mean, var):
        """
        Computes Gaussian probability density function for value x.
        """
        eps = 1e-6  # To prevent division by zero
        coeff = 1.0 / np.sqrt(2.0 * np.pi * var + eps)
        exponent = np.exp(- (x - mean) ** 2 / (2 * var + eps))
        return coeff * exponent



    def predict(self, X):
        """
        Predicts class labels for the input data X.
        """
        predictions = []
        for x in X:
            class_probs = {}
            for cls in self.class_priors:
                # Start with log prior
                log_prob = np.log(self.class_priors[cls])
                
                # Add log likelihoods for each feature
                for idx in range(len(x)):
                    mean = self.means[cls][idx]
                    var = self.variances[cls][idx]
                    prob = self.gaussian_probability(x[idx], mean, var)
                    log_prob += np.log(prob + 1e-6)  # add epsilon to avoid log(0)
                
                class_probs[cls] = log_prob  # Store total log probability for class
                
            # Pick class with highest log probability
            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)
        return np.array(predictions)


    def predict_proba(self, X):
        """
        Returns probability of each class for each sample in X.
        """
        prob_results = []
        for x in X:
            class_probs = {}
            total_prob = 0
            for cls in self.class_priors:
                prob = np.log(self.class_priors[cls])
                for idx in range(len(x)):
                    mean = self.means[cls][idx]
                    var = self.variances[cls][idx]
                    likelihood = self.gaussian_probability(x[idx], mean, var)
                    prob += np.log(likelihood + 1e-6)
                class_probs[cls] = np.exp(prob)
                total_prob += np.exp(prob)
            # Normalize probabilities
            for cls in class_probs:
                class_probs[cls] /= total_prob
            prob_results.append(class_probs)
        return prob_results


In [20]:
class MultinomialNaiveBayes:
    def __init__(self, alpha=1.0):
        """
        Initialize the Multinomial Naive Bayes classifier.
        
        Parameters:
        alpha : Laplace smoothing factor (default=1.0)
        """
        self.alpha = alpha               # Smoothing parameter
        self.class_priors = {}           # To store P(C) for each class
        self.feature_probs = {}          # To store P(x_j|C) for each feature and class
        self.classes_ = None             # Unique class labels (set during fit)
        self.V = 0                       # Vocabulary size (number of features)


    def fit(self, X, y):
        """
        Train the model on the training data.
        
        Parameters:
        X : ndarray of shape (m, n), where m = number of samples, n = number of features
        y : ndarray of shape (m,), target labels
        """
        X = np.array(X)
        y = np.array(y)
        m, n = X.shape
        self.V = n  # Number of features (vocabulary size)
        self.classes_, class_counts = np.unique(y, return_counts=True)
        
        # Calculate class priors P(C)
        for cls, count in zip(self.classes_, class_counts):
            self.class_priors[cls] = count / m
        
        # Calculate likelihoods P(x_j|C) with Laplace smoothing
        for cls in self.classes_:
            X_cls = X[y == cls]  # Filter samples of this class
            feature_counts = np.sum(X_cls, axis=0)  # Sum feature occurrences
            total_feature_count = np.sum(feature_counts)
            
            # Apply Laplace smoothing
            probs = (feature_counts + self.alpha) / (total_feature_count + self.alpha * self.V)
            
            self.feature_probs[cls] = probs  # Store probabilities for this class


    def predict_proba(self, X):
            """
            Predict probabilities for each class for the input samples.
            
            Parameters:
            X : ndarray of shape (m, n), input samples
            
            Returns:
            probabilities : ndarray of shape (m, len(classes)), predicted probabilities
            """
            X = np.array(X)
            m = X.shape[0]
            probabilities = []
            
            for x in X:
                class_log_probs = {}
                for cls in self.classes_:
                    # Start with log(P(C))
                    log_prob = np.log(self.class_priors[cls])
                    
                    # Add log(P(x_j|C)) weighted by feature counts x_j
                    log_prob += np.sum(x * np.log(self.feature_probs[cls] + 1e-9))  # +1e-9 for log(0)
                    
                    class_log_probs[cls] = log_prob
                
                # Convert log probabilities to actual probabilities
                max_log = max(class_log_probs.values())
                exp_probs = {cls: np.exp(log_p - max_log) for cls, log_p in class_log_probs.items()}
                total_prob = sum(exp_probs.values())
                normalized_probs = [exp_probs[cls] / total_prob for cls in self.classes_]
                probabilities.append(normalized_probs)
            
            return np.array(probabilities)

    def predict(self, X):
        """
        Predict class labels for the input samples.
        
        Parameters:
        X : ndarray of shape (m, n), input samples
        
        Returns:
        predictions : ndarray of shape (m,), predicted class labels
        """
        probs = self.predict_proba(X)
        class_indices = np.argmax(probs, axis=1)
        return self.classes_[class_indices]




In [21]:
# Example training data
X_train = [
    [3, 0, 1],
    [2, 0, 2],
    [0, 4, 0],
    [0, 3, 1]
]
y_train = [1, 1, 0, 0]

# Example test data
X_test = [
    [1, 1, 0],
    [0, 2, 1]
]

# Create and train model
model = MultinomialNaiveBayes(alpha=1.0)
model.fit(X_train, y_train)

# Predict
print("Predicted Classes:", model.predict(X_test))
print("Predicted Probabilities:\n", model.predict_proba(X_test))


Predicted Classes: [0 0]
Predicted Probabilities:
 [[0.57142857 0.42857143]
 [0.96969697 0.03030303]]


In [ ]:
# Example dataset
X_train = np.array([[1.0], [2.0], [1.5], [5.0], [6.0], [5.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

X_test = np.array([[1.2], [5.8]])

# Train & predict
model = GaussianNaiveBayes()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)

print("Predicted Classes:", predictions)
print("Predicted Probabilities:", probabilities)


Predicted Classes: [0 1]
Predicted Probabilities: [{0: 0.9999986594824308, 1: 1.3405175691438863e-06}, {0: 1.3405175691438857e-06, 1: 0.9999986594824308}]
